![](logo1.jpg)

 ______  ___     ______                        _________
___   |/  /________  /_______ _______ ______________  /
__  /|_/ /_  __ \_  __ \  __ `/_  __ `__ \  _ \  __  / 
_  /  / / / /_/ /  / / / /_/ /_  / / / / /  __/ /_/ /  
/_/  /_/  \____//_/ /_/\__,_/ /_/ /_/ /_/\___/\__,_/   
                                                       
_____________ _________      ___________        _________            
___    |__  /_______  /_____ ___  /__  /_______ ______  /____________
__  /| |_  __ \  __  /_  __ `/_  /__  //_/  __ `/  __  /_  _ \_  ___/
_  ___ |  /_/ / /_/ / / /_/ /_  / _  ,<  / /_/ // /_/ / /  __/  /    
/_/  |_/_.___/\__,_/  \__,_/ /_/  /_/|_| \__,_/ \__,_/  \___//_/     
                                                                     
 
                                                                        
                                                                       
_______________________________________________________________________
_/_____//_____//_____//_

# **shAI Training 2023 | Level 1**

## Task #8 (End-to-End ML Project {part_2})

## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [3]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)
   
fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(housing_prepared, housing_labels, random_state=42)

# 1- Select and Train a Model

# Let’s first train a LinearRegression model 

In [5]:
# CODE HERE
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

# First try it out on a few instances from the training set:


In [6]:
some_data = X_test[:5]
some_labels = y_test[:5]

In [7]:
# CODE HERE

predictions = lin_reg.predict(some_data)

for output, prediction in zip(some_labels, predictions):
    print(f"Model prediction = {str(round(prediction))} | Actual value = {str(round(output))} ")

Model prediction = 137958 | Actual value = 96700 
Model prediction = 131237 | Actual value = 75500 
Model prediction = 303020 | Actual value = 430900 
Model prediction = 224187 | Actual value = 332600 
Model prediction = 348398 | Actual value = 500001 


# measure this regression model’s RMSE on the whole training set 
* sing Scikit-Learn’s mean_squared_error() function:

In [8]:
from sklearn.metrics import mean_squared_error

In [9]:
# CODE HERE

housing_predictions = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

67617.33196607907


# judge on the RMSE result for this model 
write down your answar 

A high RMSE suggests that the model's predictions are far from the actual values on average, indicating poor performance.

I think it can be optimized a little bit

# Let’s train a Decision Tree Regressor model 
## more powerful model

In [10]:
from sklearn.tree import DecisionTreeRegressor 

In [11]:
# CODE HERE

tree_reg = DecisionTreeRegressor(random_state = 42)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

# Now evaluate the model on the training set 
* using Scikit-Learn’s mean_squared_error() function:

In [12]:
# CODE HERE

housing_predictions2 = tree_reg.predict(X_train)
lin_mse2 = mean_squared_error(y_train, housing_predictions2)
lin_rmse2 = np.sqrt(lin_mse2)
print(lin_rmse2)

0.0


# Explaine this result 
write down your answar

Overfitting!!

Overfitting: Decision trees, especially if they're deep, have a tendency to overfit the training data. This means the model captures the noise in the training data rather than the underlying pattern. Consequently, it can perfectly predict the training data but perform poorly on unseen data. The RMSE being 0 indicates perfect predictions on the training set, but it might not generalize well to new data.

In [13]:
# If i tried to test the model on a data that the model never seen before, The RMSE will differ like that

housing_predictions3 = tree_reg.predict(X_test)
lin_mse3 = mean_squared_error(y_test, housing_predictions3)
lin_rmse3 = np.sqrt(lin_mse3)
print(lin_rmse3)

71517.32474346654


# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [14]:
from sklearn.model_selection import cross_val_score

In [15]:
# CODE HERE

tree_model = DecisionTreeRegressor()

tree_scores = cross_val_score(tree_model, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(np.abs(tree_scores))

2- display the resultant scores and calculate its Mean and Standard deviation

In [16]:
# CODE HERE

print("Decision Tree RMSE scores:")
for index, score in enumerate(tree_rmse_scores, 1):
    print(index, f" : {round(score, 2)}")

print("\nRMSE Mean :", round(tree_rmse_scores.mean(), 2))

print("RMSE Std  :", round(tree_rmse_scores.std(), 2))

Decision Tree RMSE scores:
1  : 71412.32
2  : 70377.2
3  : 78563.2
4  : 70883.39
5  : 65604.47
6  : 80048.32
7  : 70138.44
8  : 69701.6
9  : 68034.96
10  : 68123.68

RMSE Mean : 71288.76
RMSE Std  : 4329.16


3-repaet the same steps to compute the same scores for the Linear Regression  model 

*notice the difference between the results of the two models*

In [17]:
# CODE HERE

linear_model = LinearRegression()

linear_scores = cross_val_score(linear_model, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
linear_rmse_scores = np.sqrt(np.abs(linear_scores))

In [18]:
print("Linear Regression RMSE scores:")
for index, score in enumerate(linear_rmse_scores, 1):
    print(index, f" : {round(score, 2)}")

print("\nRMSE Mean :", round(linear_rmse_scores.mean(), 2))

print("RMSE Std  :", round(linear_rmse_scores.std(), 2))

Linear Regression RMSE scores:
1  : 68197.29
2  : 249437.33
3  : 66920.17
4  : 66377.69
5  : 63339.96
6  : 71755.04
7  : 65900.12
8  : 74877.21
9  : 66411.72
10  : 66863.08

RMSE Mean : 86007.96
RMSE Std  : 54563.55


## Let’s train one last model the RandomForestRegressor.

In [19]:
# CODE HERE

from sklearn.ensemble import RandomForestRegressor

forest_model = RandomForestRegressor()
forest_scores = cross_val_score(forest_model, X_train, y_train, scoring = "neg_mean_squared_error", cv = 10)
forest_rmse_scores = np.sqrt(np.abs(forest_scores))

# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [20]:
# CODE HERE

print("Random Forest RMSE scores:")
for index, score in enumerate(forest_rmse_scores, 1):
    print(index, f" : {round(score, 2)}")

print("\nRMSE Mean :", round(forest_rmse_scores.mean(), 2))

print("RMSE Std  :", round(forest_rmse_scores.std(), 2))

Random Forest RMSE scores:
1  : 50338.55
2  : 51811.25
3  : 50662.83
4  : 51926.09
5  : 46460.65
6  : 55167.45
7  : 50232.74
8  : 51008.23
9  : 50215.77
10  : 49813.02

RMSE Mean : 50763.66
RMSE Std  : 2051.69


# Save every model you experiment with 
*using the joblib library*

In [21]:
# CODE HERE

import joblib

joblib.dump(linear_model, 'LinearRegression_model.pkl')
joblib.dump(tree_model,   'DecisionTree_model.pkl')
joblib.dump(forest_model, 'RandomForest_model.pkl')

['RandomForest_model.pkl']

## Now you have a shortlist of promising models. You now need to fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor 
*It may take a long time*

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
# CODE HERE

param_grid = [
              {'n_estimators': [3, 5, 10, 20, 30], 'max_features':[4, 5, 6, 7, 8]},
              {'bootstrap':[False], 'max_features':[2, 3, 4],'n_estimators':[3, 10]}
]

forest_reg = RandomForestRegressor(random_state = 42)

grid_search = GridSearchCV(forest_reg, param_grid, cv = 5, scoring = 'neg_mean_squared_error', return_train_score = True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [4, 5, 6, 7, 8],
                          'n_estimators': [3, 5, 10, 20, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

with the evaluation scores

In [24]:
# CODE HERE

cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
    print(round(np.sqrt(np.abs(mean_score)), 2), params)

61025.88 {'max_features': 4, 'n_estimators': 3}
56388.55 {'max_features': 4, 'n_estimators': 5}
52948.7 {'max_features': 4, 'n_estimators': 10}
51348.75 {'max_features': 4, 'n_estimators': 20}
51052.07 {'max_features': 4, 'n_estimators': 30}
59953.22 {'max_features': 5, 'n_estimators': 3}
56379.74 {'max_features': 5, 'n_estimators': 5}
53666.31 {'max_features': 5, 'n_estimators': 10}
51786.05 {'max_features': 5, 'n_estimators': 20}
51264.24 {'max_features': 5, 'n_estimators': 30}
58607.04 {'max_features': 6, 'n_estimators': 3}
54997.14 {'max_features': 6, 'n_estimators': 5}
52162.56 {'max_features': 6, 'n_estimators': 10}
50848.42 {'max_features': 6, 'n_estimators': 20}
50600.31 {'max_features': 6, 'n_estimators': 30}
59396.93 {'max_features': 7, 'n_estimators': 3}
55799.64 {'max_features': 7, 'n_estimators': 5}
53043.29 {'max_features': 7, 'n_estimators': 10}
51032.06 {'max_features': 7, 'n_estimators': 20}
50735.29 {'max_features': 7, 'n_estimators': 30}
59048.57 {'max_features': 8, 

# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [25]:
# CODE HERE

feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances


# array([6.84493392e-02, 6.49131340e-02, 4.17428333e-02, 1.45158216e-02,
#        1.37060650e-02, 1.43001651e-02, 1.29591331e-02, 3.71833888e-01,
#        4.94502910e-02, 1.09758357e-01, 6.11769498e-02, 7.39554036e-03,
#        1.65012599e-01, 2.28668090e-04, 1.83994495e-03, 2.71727020e-03])

array([6.66276481e-02, 6.07537030e-02, 4.03096731e-02, 1.52423600e-02,
       1.47674103e-02, 1.46739077e-02, 1.37022307e-02, 3.78997134e-01,
       5.09914428e-02, 1.09765729e-01, 5.55973364e-02, 6.72479550e-03,
       1.66078781e-01, 2.32494481e-04, 2.18657403e-03, 3.34878064e-03])

2-display these importance scores next to their corresponding attribute names:

In [26]:
# CODE HERE

num_attribs = list(train_set.drop(["median_house_value"],axis = 1).columns)
extra_attribs = ["population_per_household", "bedroom_per_room", "rooms_per_household"]
cat_one_hot_attribs = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse = True)

[(0.3789971339914692, 'median_income'),
 (0.16607878076062677, '<1H OCEAN'),
 (0.1097657285841296, 'population_per_household'),
 (0.06662764807490762, 'longitude'),
 (0.060753703027052984, 'latitude'),
 (0.055597336394445, 'bedroom_per_room'),
 (0.050991442767811965, 'ocean_proximity'),
 (0.04030967310602871, 'housing_median_age'),
 (0.01524235997423226, 'total_rooms'),
 (0.014767410308921418, 'total_bedrooms'),
 (0.014673907701135872, 'population'),
 (0.013702230656568464, 'households'),
 (0.0067247955017928065, 'rooms_per_household'),
 (0.00334878064431095, 'NEAR BAY'),
 (0.0021865740252877777, 'ISLAND'),
 (0.0002324944812786841, 'INLAND')]

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

- Get the predictors and the labels from your test set
- Run your full_pipeline to transform the data

In [27]:
# CODE HERE

# The test set was generated after transforming the whole dataset so this so this point is already done at CODE CELL num 3

- Evaluate the final model on the test set

In [28]:
# CODE HERE

best_model = grid_search.best_estimator_
best_predictions = best_model.predict(X_test)
mse = mean_squared_error(y_test, best_predictions)
rmse = np.sqrt(np.abs(mse))
rmse

49611.310138215784

# Compute a 95% confidence interval for the generalization error 
*using scipy.stats.t.interval():*

In [29]:
from scipy import stats

In [30]:
# CODE HERE

confidence = 0.95
squared_errors = (best_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
loc = squared_errors.mean(), scale = stats.sem(squared_errors)))

array([47467.12510437, 51666.58709056])

# Great Job!
# #shAI_Club